In [1]:
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
!pip install onedrivedownloader
!pip install torch_geometric==2.3.1
!pip install timm==0.9.7
!pip install gradio

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 0.9.10
    Uninstalling timm-0.9.10:
      Successfully uninstalled timm-0.9.10


In [2]:
from onedrivedownloader import download
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from timm import create_model
import torch
import pandas as pd
import torch
import joblib
import warnings
import os
import shutil
warnings.filterwarnings('ignore')

In [3]:
data_url = 'https://unibari-my.sharepoint.com/:u:/g/personal/raffaele_scaringi_uniba_it/Ef0Fl3y9aa1Nid8WrATHcRIBLRkUIUxKE_eR0iPBhxlVEQ?e=e3vq0a'
destination_dir = 'data'
download(url=data_url, filename='file.zip', unzip=True, unzip_path=destination_dir)

Unzipping file...


Extracting files: 100%|██████████| 142/142 [00:00<00:00, 23583.10it/s]


'data'

In [4]:
destination_dir += "/data"

In [5]:
for f in os.listdir(f"{destination_dir}/code"):
    shutil.move(f"{destination_dir}/code/{f}", f"{f}")

In [6]:
data = torch.load(f'{destination_dir}/dataset/common_dataset/train_data.pt').to('cuda')

In [7]:
data

HeteroData(
  artwork={ x=[116475, 128] },
  artist={ x=[2501, 1] },
  gallery={ x=[1099, 1] },
  city={ x=[596, 1] },
  country={ x=[58, 1] },
  style={ x=[32, 1] },
  period={ x=[186, 1] },
  genre={ x=[18, 1] },
  serie={ x=[823, 1] },
  tag={ x=[5424, 1] },
  media={ x=[167, 1] },
  subject={ x=[6985, 1] },
  training_node={ x=[268, 1] },
  field={ x=[54, 1] },
  movement={ x=[243, 1] },
  people={ x=[109, 1] },
  emotion={ x=[9, 1] },
  (artist, belongstofield, field)={ edge_index=[2, 987] },
  (artist, belongstomovement, movement)={ edge_index=[2, 1056] },
  (artist, haspatron, people)={ edge_index=[2, 124] },
  (artist, hassubject, subject)={ edge_index=[2, 21054] },
  (artist, relatedtoschool, training_node)={ edge_index=[2, 498] },
  (artist, trainedby, artist)={ edge_index=[2, 47] },
  (artwork, about, tag)={ edge_index=[2, 181644] },
  (artwork, createdby, artist)={ edge_index=[2, 52188] },
  (artwork, elicit, emotion)={ edge_index=[2, 52188] },
  (artwork, hasgenre, genre)=

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load(f'{destination_dir}/model/best_new_multitask_model.pt')
feature_extractor = torch.load(f'{destination_dir}/model/vit_fine_tune_style.pt')
pca = joblib.load(f'{destination_dir}/model/pca.pk')

In [9]:
model

NewMultiTaskClassificationModel(
  (encoder_style): MultiGNNEncoder(
    (activation): Tanh()
    (encoders): ModuleDict(
      (0): HeteroConv(num_relations=1)
    )
  )
  (encoder_genre): MultiGNNEncoder(
    (activation): Tanh()
    (encoders): ModuleDict(
      (0): HeteroConv(num_relations=1)
    )
  )
  (encoder_emotion): MultiGNNEncoder(
    (activation): Tanh()
    (encoders): ModuleDict(
      (0): HeteroConv(num_relations=1)
    )
  )
  (mlp): Sequential(
    (0): Linear(in_features=7680, out_features=3840, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=3840, out_features=1920, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=1920, out_features=960, bias=True)
    (7): ReLU(inplace=True)
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=960, out_features=480, bias=True)
    (10): ReLU(inplace=True)
    (11): Dropout(p=0.2, inplace=False)
  )
  (

In [10]:
feature_extractor

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [11]:
feature_extractor.reset_classifier(num_classes=0)

In [12]:
class PredictionPipeline():
    def __init__(self, model, feature_extractor, pca, graph, preprocessing, mapping_dir):
        self.model = model
        self.feature_extractor = feature_extractor
        self.graph = graph
        self.preprocessing = preprocessing
        self.pca = pca
        self.mapping_dir = mapping_dir

    def map_labels(self, style, genre, emotion):
        style2name = pd.read_csv(f"{self.mapping_dir}/style_entidx2name.csv", names=["idx", "name"]).drop(0)
        style2name["idx"] -= 1
        style2name.reset_index(inplace=True, drop=True)
        genre2name = pd.read_csv(f"{self.mapping_dir}/genre_entidx2name.csv", names=["idx", "name"])
        emotion2name = pd.read_csv(f"{self.mapping_dir}/emotion_entidx2name.csv", names=["idx", "name"])
        return style2name.loc[style, "name"], genre2name.loc[genre, "name"], emotion2name.loc[emotion, "name"]

    def predict(self, image):
        tensor = self.preprocessing(image.convert('RGB')).unsqueeze(dim=0).to('cuda')
        features = self.pca.transform(self.feature_extractor(tensor).cpu().numpy())
        ans = self.model(torch.from_numpy(features).to('cuda'), self.graph.x_dict, self.graph.edge_index_dict)
        style, genre, emotion = ans
        style, genre, emotion = torch.argmax(style).item(), torch.argmax(genre).item(), torch.argmax(emotion).item()
        style, genre, emotion = self.map_labels(style, genre, emotion)
        return f"The artwork is in {style} style, represents a {genre}, and can evoke {emotion}."

In [13]:
preprocessing = Compose([
    ToTensor(),
    Resize((224, 224)),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
pipeline = PredictionPipeline(model,
                              feature_extractor,
                              pca,
                              data,
                              preprocessing,
                              f"{destination_dir}/dataset/artgraph2bestemotions/mapping"
                             )

In [ ]:
import gradio as gr
gr.Interface(
    pipeline.predict,
    title="Artwork Attribtue Recognition with KGs",
    inputs=[
        gr.Image(sources=['upload'], type='pil')
    ],
    outputs=[
        gr.Textbox()
    ]
).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cf0fce75b7528f813a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
